In [1]:
import pandas as pd

In [6]:
# Function to generate banking data and save it
def generate_banking_data():
    # Sample account-level data
    account_data = {
        'customer_name': ['Alice', 'Bob', 'Charlie', 'David'],
        'customer_id': [1, 2, 3, 4],
        'pr_number': ['PR001', 'PR002', 'PR003', 'PR004'],
        'income': [50000, 60000, 70000, 80000],
        'age': [30, 40, 35, 50]
    }

# Sample transaction-level data
    transaction_data = {
        'ucic_id': [101, 102, 103, 104, 105, 106],
        'customer_id': [1, 1, 2, 3, 4, 4],
        'category': ['electronics', 'investments', 'investments', 'groceries', 'investments', 'electronics'],
        'transaction_amount': [200, 1500, 2000, 300, 2500, 400],
        'credit_debit': ['debit', 'credit', 'credit', 'debit', 'credit', 'debit']
    }
# Create DataFrames
    account_df = pd.DataFrame(account_data)
    transaction_df = pd.DataFrame(transaction_data)

    # Save data to CSV for later use
    account_df.to_csv('account_data.csv', index=False)
    transaction_df.to_csv('transaction_data.csv', index=False)

    return account_df, transaction_df

In [7]:
# Generate and save the data
account_df, transaction_df = generate_banking_data()

In [8]:
# Load saved data
account_df = pd.read_csv('account_data.csv')
transaction_df = pd.read_csv('transaction_data.csv')

In [9]:
account_df.head()
transaction_df.head()

,ucic_id,customer_id,category,transaction_amount,credit_debit
0,101,1,electronics,200,debit
1,102,1,investments,1500,credit
2,103,2,investments,2000,credit
3,104,3,groceries,300,debit
4,105,4,investments,2500,credit


In [33]:
from crewai import Agent,Task


In [12]:
from transformers import pipeline

In [14]:
# Load the Groq/Llama-3-Groq-8B-Tool-Use model from Hugging Face

# llm_model = pipeline(model="Groq/Llama-3-Groq-8B-Tool-Use")


In [17]:
llm_model = pipeline('text-generation',model="openai-community/gpt2")


model.safetensors:  19%|#9        | 105M/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

C:\Users\sudes\AppData\Roaming\Python\Python311\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [25]:
# Task definitions for the agent
def filter_investments(transaction_df):
    """Task: Filter and return investment transactions."""
    investment_transactions = transaction_df[transaction_df['category'] == 'investments']
    return investment_transactions
def summarize_investments(investment_transactions):
    """Task: Summarize investment transactions by customer."""
    investment_summary = investment_transactions.groupby('customer_id').sum(numeric_only=True)
    return investment_summary
def compare_with_peers(investment_summary, account_df):
    """Task: Merge investment data with account-level data for comparison."""
    comparison = investment_summary.merge(account_df, on='customer_id', how='left')
    return comparison

In [39]:
# Initialize the first agent: Banking Data Analyst using Hugging Face model
banking_analyst = Agent(
    role='Banking Data Analyst',
    goal='Analyze customer transactions and identify key investment patterns',
    verbose=True,
    memory=True,
    backstory="An expert in banking and customer data, focused on extracting valuable insights from transactions.",
    tools=None,  # No tools required, logic is defined within the agent
    allow_delegation=False,
    llm=llm_model
)

# Initialize the second agent: Peer Comparison Analyst using Hugging Face model
report_writer = Agent(
    role='Report Writer',
    goal='Compare customer investments with peers to evaluate financial strategies',
    verbose=True,
    memory=True,
    backstory=(
        "A financial comparison expert specializing in evaluating customers' "
        "investment strategies by comparing them with their peers."
    ),
    tools=None,  # No tools required, logic is defined within the agent
    allow_delegation=False,
    llm=llm_model  # Use Hugging Face Ollama model as the LLM
)

In [27]:
# Main function to perform the end-to-end analysis
def perform_analysis(account_df, transaction_df):
    # Step 1: Filter investments
    investment_transactions = filter_investments(transaction_df)

    # Step 2: Summarize investment data
    investment_summary = summarize_investments(investment_transactions)

    # Step 3: Compare investment data with peers
    investment_comparison = compare_with_peers(investment_summary, account_df)

    return investment_comparison

In [28]:
# Example usage
account_df, transaction_df = generate_banking_data()
investment_comparison = perform_analysis(account_df, transaction_df)

In [29]:
print("\nInvestment Comparison with Peers:")
print(investment_comparison)


Investment Comparison with Peers:
   customer_id  ucic_id  transaction_amount customer_name pr_number  income  \
0            1      102                1500         Alice     PR001   50000   
1            2      103                2000           Bob     PR002   60000   
2            4      105                2500         David     PR004   80000   

   age  
0   30  
1   40  
2   50  


In [36]:
# Task for banking data analysis
def perform_banking_analysis(account_df, transaction_df):
    analysis_task = Task(
        description="Analyze customer transactions and identify key investment patterns.",
        expected_output="A detailed report summarizing key insights from banking data, focusing on investment transactions.",
        tools=None,  # No tools required
        agent=banking_analyst
    )
# Sample analysis logic to filter and summarize investment transactions
    investment_transactions = transaction_df[transaction_df['category'] == 'investments']
    investment_summary = investment_transactions.groupby('customer_id').sum(numeric_only=True)

    # Generate a summary using the LLM
    analysis_report = analysis_task.agent.llm(f"Analyze the following investment transactions and provide insights:\n{investment_summary}")
    
    return analysis_report

In [40]:
# Task for writing a report based on analysis
def perform_report_writing(analysis_report):
    write_task = Task(
        description="Summarize the analysis into a comprehensive banking data report.",
        expected_output="A detailed report summarizing the key insights from the analysis of customer investment data.",
        tools=None,  # No tools required
        agent=report_writer,
        async_execution=False,
        output_file='banking_data_report.md'  # Save the output to a markdown file
    )
    # Generate the report using the LLM
    report_content = write_task.agent.llm(f"Write a detailed report based on the following analysis:\n{analysis_report}")

    # Save the report to a file
    with open(write_task.output_file, 'w') as file:
        file.write(report_content)
    
    return report_content

In [41]:
# Example usage
account_df, transaction_df = generate_banking_data()
analysis_report = perform_banking_analysis(account_df, transaction_df)
final_report = perform_report_writing(analysis_report)

TypeError: 'LLM' object is not callable